# How To Run
- First Log to Hug Face
- Seond Download The dataset
- Third Clone the repo from github
- Install all the packages
- Then train from scratch until session is ended
- After that resume training from the last checkpoint uploaded to hugging face
- Repeat until full train

In [ ]:
## Change with you name and member to save you name directory
hf_train_prefix = "" # example "member_3_rehab_hamdy/classes_26_30/training"
hf_test_prefix  = "" # example"member_3_rehab_hamdy/classes_26_30/test"

# Login to Hug Face

In [ ]:
from huggingface_hub import login
login("") # put you token here



# Download Dataset

In [ ]:
!pip install -q huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download
import os

# Save to Kaggle TEMP instead of WORKING
full_dir = "/kaggle/temp/LaSOT_partial"

categories =  [
    # put your name of classes here to download
]

# Example 
# categories =  [
#     "gametarget",
#     "gecko",
#     "giraffe",
#     "goldfish",
#     "gorilla"
# ]

# You can uncomment or add more later if needed

os.makedirs(full_dir, exist_ok=True)

for cls in categories:
    save_path = os.path.join(full_dir, cls)
    os.makedirs(save_path, exist_ok=True)
    print(f"🚀 Start downloading: {cls}")

    snapshot_download(
        repo_id="l-lt/LaSOT",
        repo_type="dataset",
        local_dir=save_path,
        allow_patterns=[f"{cls}.zip"]
    )

    print(f"✅ Finished downloading: {cls}")

print("🎉 All requested categories downloaded to /kaggle/temp/LaSOT_partial")


## Unziping the data

In [ ]:
import os
import zipfile

base_dir = "/kaggle/temp/LaSOT_partial"

for cls in os.listdir(base_dir):
    cls_dir = os.path.join(base_dir, cls)
    if not os.path.isdir(cls_dir):
        continue

    # Find all zip files in this class folder
    for file in os.listdir(cls_dir):
        if file.endswith(".zip"):
            zip_path = os.path.join(cls_dir, file)
            print(f"📦 Unzipping {zip_path} ...")

            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(cls_dir)
                print(f"✅ Extracted: {cls}")

                # Delete the zip file to save space
                os.remove(zip_path)
                print(f"🗑️ Deleted: {zip_path}")

            except Exception as e:
                print(f"❌ Error with {zip_path}: {e}")

print("🎉 All zips extracted and deleted successfully.")

In [ ]:
%cd /kaggle/temp/LaSOT_partial/
%ls

# Clone the repo from github

In [ ]:
%cd /kaggle/working
!rm -rf seqtrack5 && git clone https://github.com/Abdo0213/seqtrack5

In [ ]:
import os

# Define target path
save_dir = "/kaggle/working/seqtrack5/SeqTrack"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "selected_classes.txt")

# --- Define the content of your file here ---
# Replace this with the actual classes you need, one per line
classes_content = """
# put the name of your classes here
"""
# ---------------------------------------------

# Save it as selected_classes.txt
try:
    with open(save_path, "w") as f:
        f.write(classes_content.strip()) # .strip() removes leading/trailing whitespace

    # Set environment variable
    os.environ["LASOT_CLASSES_FILE"] = save_path

    print(f"✅ File saved as: {save_path}")
    print(f"Environment variable set: LASOT_CLASSES_FILE={os.environ['LASOT_CLASSES_FILE']}")
except Exception as e:
    print(f"❌ An error occurred while creating the file: {e}")

In [ ]:
# --- Read and display the file content ---
try:
    with open(save_path, "r") as f:
        file_content = f.read()

    print("\n📄 Contents of selected_classes.txt:")
    print("------------------------------------")
    print(file_content)
except Exception as e:
    print(f"❌ An error occurred while reading the file: {e}")


In [ ]:
## See inside the repo
%cd /kaggle/working/seqtrack5/SeqTrack
!python checkRepo.py 

# Install needed Pckages

In [ ]:
%cd /kaggle/working/seqtrack5/SeqTrack
# !pip install -r requirements.txt
!pip install -r installed_packages.txt
!pip install jpeg4py

In [ ]:
!pip install --upgrade protobuf==3.20 tensorboard==2.14.0

# Train From scratch

In [ ]:
%cd /kaggle/working/seqtrack5/SeqTrack
!python tracking/train.py --script seqtrack --config seqtrack_b256 --save_dir . --mode single --seed 3 --phase phase_1 --repo_id ayamohamed2500/assignment_5 --hf_train_prefix {hf_train_prefix} | tee train_log.txt

# Only copy if file exists
import os
if os.path.exists("train_log.txt"):
    !mkdir -p /kaggle/working/outputs
    !cp train_log.txt /kaggle/working/outputs/
    print("File copied to outputs folder!")
else:
    print("train_log.txt does not exist.")


# Train from specific checkpoint

In [ ]:
%cd /kaggle/working/seqtrack5/SeqTrack
!python tracking/train.py --script seqtrack --config seqtrack_b256 --repo_id ayamohamed2500/assignment_5 --mode single --phase phase_1 --hf_train_prefix {hf_train_prefix} --resume hf://ayamohamed2500/assignment_5/{hf_train_prefix}/SEQTRACK_ep0020.pth.tar

# Test from scratch

In [ ]:
!pip install lmdb
!pip install tikzplotlib

In [ ]:
%cd /kaggle/working/seqtrack5/SeqTrack
!python tracking/assignment_4_test.py --repo_id ayamohamed2500/assignment_5 --phase {hf_train_prefix} --start_epoch 1 --end_epoch 100 --upload_prefix {hf_test_prefix}

# Resuming test from checkpoint

In [ ]:
%cd /kaggle/working/seqtrack5/SeqTrack
!python tracking/assignment_4_test.py --repo_id ayamohamed2500/assignment_5 --phase {hf_train_prefix} --start_epoch 1 --end_epoch 100 --upload_prefix {hf_test_prefix} --resume_epoch 11   # example: skip epochs < 11